In [1]:
import coremltools
from PIL import Image
import os
import re
import shutil
import json
import yaml

## Open source model

In [7]:
model_folder = '/Users/landyrev/Downloads/latest'

model_name = filter(
    lambda x: re.match(r'.*.caffemodel', x, re.IGNORECASE), 
    os.listdir(model_folder)
)[0]

model_path = os.path.join(model_folder, model_name)

## Load and convert model

In [4]:
m = coremltools.converters.caffe.convert(
    (
        model_path, os.path.join(model_folder, 'deploy.prototxt'), 
        os.path.join(model_folder, 'mean.binaryproto')
    ),
    class_labels=os.path.join(model_folder, 'labels.txt'), 
    image_input_names='data'
)

## Add model metadata

In [5]:
m.author = 'Aleksey Landyrev'
m.short_description = 'How to draw model'
m.save('realmodel.mlmodel')

## Copy model into target project

In [12]:
shutil.copy('./realmodel.mlmodel', '../htd/Data/realmodel.mlmodel')

## Copy needed images

In [27]:
tutorials = []

shutil.rmtree('../../htd/Data/Images.bundle')
os.mkdir('../../htd/Data/Images.bundle')

for label in open(os.path.join(model_folder, 'labels.txt')):
    label = label[:-1]
    if label == "random":
        continue
    
    src_dir = os.path.join('../AllImages/', label)
    dst_dir = os.path.join('../../htd/Data/Images.bundle/', label)
    meta_file = os.path.join(src_dir, 'meta.yaml')
    
    os.mkdir(dst_dir)
    
    for image in os.listdir(src_dir):
        if not re.match(r'.*(\.png|\.jpg)', image, re.IGNORECASE):
            continue
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(dst_dir, label + "_" + image)
        )
    
    tutorial = {
        "name": label,
        "preview": os.path.join(label, label+ '_preview.png'),
        "tutorials": sorted([os.path.join(label, x) 
             for x in os.listdir(dst_dir)
             if x.endswith('.jpg')
            ])
    }
    print tutorial
    tutorial.update(yaml.load(open(meta_file)))
    tutorials.append(tutorial)

yaml.dump(
    tutorials,
    open("../../htd/Data/Images.bundle/levels.yaml", "w")
)

{'tutorials': ['lutz_boat0/lutz_boat0_00.jpg', 'lutz_boat0/lutz_boat0_01.jpg', 'lutz_boat0/lutz_boat0_02.jpg', 'lutz_boat0/lutz_boat0_03.jpg', 'lutz_boat0/lutz_boat0_04.jpg', 'lutz_boat0/lutz_boat0_05.jpg'], 'preview': 'lutz_boat0/lutz_boat0_preview.png', 'name': 'lutz_boat0'}
{'tutorials': ['lutz_lighthouse0/lutz_lighthouse0_00.jpg', 'lutz_lighthouse0/lutz_lighthouse0_01.jpg', 'lutz_lighthouse0/lutz_lighthouse0_02.jpg', 'lutz_lighthouse0/lutz_lighthouse0_03.jpg', 'lutz_lighthouse0/lutz_lighthouse0_04.jpg', 'lutz_lighthouse0/lutz_lighthouse0_05.jpg'], 'preview': 'lutz_lighthouse0/lutz_lighthouse0_preview.png', 'name': 'lutz_lighthouse0'}
{'tutorials': ['lutz_pug0/lutz_pug0_00.jpg', 'lutz_pug0/lutz_pug0_01.jpg', 'lutz_pug0/lutz_pug0_02.jpg', 'lutz_pug0/lutz_pug0_03.jpg', 'lutz_pug0/lutz_pug0_04.jpg', 'lutz_pug0/lutz_pug0_05.jpg', 'lutz_pug0/lutz_pug0_06.jpg'], 'preview': 'lutz_pug0/lutz_pug0_preview.png', 'name': 'lutz_pug0'}
{'tutorials': ['lutz_swan0/lutz_swan0_00.jpg', 'lutz_swan0/lu

## Try to predict test image

In [13]:
def predict(image_name):
    image = Image.open(image_name).convert('L')
    image = image.resize((224, 224))
    features = m.predict({
        'data': image
    }).get('softmax')
    for feature in features:
        print feature, int(features[feature] * 100)
        
predict('/Users/landyrev/Downloads/fvsdfvsdfv.jpg')

lutz_swan0 0
lutz_lighthouse0 0
random 0
lutz_boat0 99
lutz_pug0 0
lutz_woman0 0
